# 2.1.4 Aprendizaje online y gradiente descendente estocástico

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

## 2.1.4.1 Definición del problema de estimación de parámetros

Se desean encontra los parámetros w0 y w1 del siguiente modelo de regresión entre las variables x y y:

$$ y_{i} = w_{i}x_{i} + w_{0} + e_{i} $$

a partir de un conjunto de observaciones ${d_{i}, x_{i}}$, donde $d_{i}$ es el valor real (observado), $y_{i}$ es su aproximación usando el modelo de regresión descrito, y $e_{i}$ es el error aleatorio.

In [ ]:
#
# Datos del problema
#
x_sample = [
    0.6231,0.7460,0.1087,0.3654,0.3065,0.6886,
    0.4966,0.2618,0.7607,0.4424,0.7731,
]

y_sample = [
    1.6266,1.8508,0.9519,1.2828,1.2283,1.8722,
    1.4657,1.1097,1.7826,1.4267,1.8248,
]

## 2.1.4.2 Función de pérdida

$$ SSE(w_0, w_1) = \sum_{i}^{} (d_i - y_i)^2 $$

In [ ]:
#
# Sumatoria del error cuadrático o pérdida cuadrática
#
def sse(w0, w1):
    y_forecasts = [w0 + w1 * x_value for x_value in x_sample]
    errors = [y_true - y_forecast for y_true, y_forecast in zip(y_sample, y_forecasts)]
    squared_errors = [error ** 2 for error in errors]
    return sum(squared_errors)

## 2.1.4.3 Aprendizaje Batch

De acuerdo con el método del gradiente

<center>

\begin{align*}

    w_k &= w_{k-1} - \mu \frac{\delta}{\delta_w} SEE^2(\delta_{w-1})  \\
    &= w_{k-1} - \mu \frac{\delta}{\delta_w} \sum_{i}{} e_î^2
    
\end{align*}

</center>

Donde w=[$ w_0, w_1 $]

La ecuación anterior se puede reescribir como:

$$ w_{k} = w_{k-1} - \mu \sum_{i}{} (\frac{\delta}{\delta_w} e_î^2) $$

## 2.1.4.4 Aprendizaje online

$$ w_{k} = w_{k-1} - \mu (\frac{\delta}{\delta_w} e_î^2) $$

El proceso es el siguiente:

- Se computa el gradiente instantaneo para el punto i
- Se realiza la corrección del vector del parámetro $w_k$ con el gradiente en el punto $i$
- Se pasa al siguiente punto $ i = i + 1 $ y se retorna al primer paso

## 2.1.4.5 Derivadas

$$ \frac{\delta}{\delta_{w0}} e_î^2 = -2e_î $$
$$ \frac{\delta}{\delta_{w1}} e_î^2 = -2e_îx_i $$

In [ ]:
def gradient(w0, w1, x_value, y_value):

    error = y_value - w0 - w1 * x_value
    gradient_w0 = -2 * error
    gradient_w1 = -2 * error * x_value

    return gradient_w0, gradient_w1

## 2.1.4.6 Ecuación de mejora

$$ w_{k} = w_{k-1} - \mu (\frac{\delta}{\delta_w} e_î^2) $$

In [ ]:
def improve(w0, w1, mu, x_value, y_value):

    gradient_w0, gradient_w1 = gradient(w0, w1, x_value, y_value)
    w0 = w0 - mu * gradient_w0
    w1 = w1 - mu * gradient_w1

    return w0, w1

## 1.2.4.7 Proceso iterativo

In [ ]:
def plot_contour():

    W0 = np.linspace(0.0, 0.8, 50)
    W1 = np.linspace(2.4, 3.2, 50)
    W0, W1 = np.meshgrid(W0, W1)
    F = sse(W0, W1)

    levels = [0, 0.2, 0.5, 0.75, 1, 2, 3, 5, 10, 20, 40, 60]

    fig, ax = plt.subplots(figsize=(8, 8))

    ax.set_aspect("equal", "box")
    ax.contourf(W0, W1, F, cmap=cm.Greys, levels=levels, alpha=0.8)
    ax.contour(W0, W1, F, colors="gray", levels=levels)
    ax.grid()
    ax.set_xlabel("w0")
    ax.set_ylabel("w1")

In [ ]:
#
# Punto de inicio
#
w0 = 0.5
w1 = 3.0

history = {
    "w0": [w0],
    "w1": [w1],
    "sse": [sse(w0, w1)],
}

mu = 0.03

for epoch in range(10):

    for x_value, y_value in zip(x_sample, y_sample):

        w0, w1 = improve(w0, w1, mu, x_value, y_value)

        history["w0"].append(w0)
        history["w1"].append(w1)
        history["sse"].append(sse(w0, w1))

print(" w0 = {:6.4f}\n w1 = {:6.4f}\nSSE = {:6.4f}\n".format(w0, w1, sse(w0, w1)))

plot_contour()
plt.plot(history["w0"], history["w1"], marker=".", color="black")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history["sse"], ".-k")
plt.xlabel("Iteraciones")
plt.ylabel("SSE(w0, w1)")
plt.show()

In [ ]:
def epocas(w0, w1, epocas, learning_rate):
    
    history = { "w0": [w0], "w1": [w1], "sse": [sse(w0, w1)]}
    
    for epoch in range(epocas):
        for x_value, y_value in zip(x_sample, y_sample):
            
            w0, w1 = improve(w0, w1, learning_rate, x_value, y_value)

            history["w0"].append(w0)
            history["w1"].append(w1)
            history["sse"].append(sse(w0, w1))

    #
    # Ultimo resultado obtenido
    #
    # print(" w0 = {:6.4f}\n w1 = {:6.4f}\nSSE = {:6.4f}\n".format(w0, w1, sse(w0, w1)))
    
    return history

In [ ]:
history = epocas(0.5, 3.0, 10, 0.03)

In [ ]:
plot_contour()
plt.plot(history["w0"], history["w1"], marker=".", color="black")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history["sse"], ".-k")
plt.xlabel("Iteraciones")
plt.ylabel("SSE(w0, w1)")
plt.show()

In [ ]:
print('ok_')